# Table of Contents
1. [Travelling Salesman problem](#Lab-6-Optimization-Exercise:-Travelling-Salesman-problem)
2. [Multi-objective optimization](#Lab-7-Optimization-exercises:-Multi-objective-optimization)

# Lab 6 Optimization Exercise: Travelling Salesman problem

Implement the optimization algorithms on the Travelling salesman problem:
- greedy algorithm
- simulated annealing
- variable neighbourhood search
- local search

## Travelling Salesman Problem

Cities are consecutively numbered: $1,2,...,n$

We encode the solutions as $x=(x_1,x_2,...,x_n,x_1)$ where:

* $x_1$ is the index of the home city
* $x_i$ is the index of the $i^{th}$ city visited along the way
* $x_n$ is the last city visited
* Every city must be visited exactly once

In [1]:
import numpy as np
import timeit
import plotly.graph_objects as go

from numpy import sqrt
from numpy.random import Generator, PCG64

# PCG64 makes a guarantee that a fixed seed

In [2]:
def simulateCities(rg, n_cities):
    # simulates the cities' lat and lon
    cities = {}
    for j in range(n_cities):
        cities[j] = tuple(rg.random(size=2).round(2)) # we use round(2) to make the digits more human readable
    return cities

def drawSalesman(path, cities, title="Path taken"):
    # loop through the coordinates using the path sequence
    x = [cities[p][0] for p in path]  # sequence of x coordinates
    y = [cities[p][1] for p in path]  # sequence of y coordinates

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, name='path', mode='lines'))
    fig.add_trace(go.Scatter(x=x, y=y, name="cities", mode='markers+text', 
        marker={'size': 10}, text=[str(i) for i in path], textposition="bottom center",))
    fig.update_layout(template="presentation", title=title, xaxis_title="X", yaxis_title="Y", width=800, height=600)
    fig.update_xaxes(range=[0, 1])
    fig.update_yaxes(range=[0, 1])
    fig.show() # to reveal the figure
    
def evaluate(path, cities):
    # measure the total distance travelled
    distance = 0
    for n, p in enumerate(path[:-1]): # we loop until the last city -1
        x1, y1 = cities[p]
        x2, y2 = cities[path[n+1]]
        length = sqrt((x2-x1)**2 + (y2-y1)**2)  # euclidean distance
        distance += length
    
    return distance

def get_distance(city_from, city_to, cities):
        x1, y1 = cities[city_from]
        x2, y2 = cities[city_to]
        return sqrt((x1-x2)**2 + (y1-y2)**2)

In [3]:
def generateNewCitySeq_fe(rg, path):
    new_path = np.array(path.copy()) # make a copy of path, convert it to array for the next method (shuffle)
    # note that path.copy() generates a DEEP copy of the path given
    rg.shuffle(new_path[1:-1])
    return list(new_path)

def FullEnumeration(rg, cities, iterations):
    tracker = [] # a list of tuple to track the min distance
    path = list(range(len(cities))) + [0]  # initial solution
    distance_travelled = evaluate(path, cities) # calculate the inital objective function
    
    min_distance_travelled = distance_travelled # save the current solution as the "best solution"
    best_solution = path                        # save the current solution as the "best solution"
    
    print("running algorithm...")
    start_time = timeit.default_timer() # implement a timer to track running time
    for i in range(1, iterations):
        path = generateNewCitySeq_fe(rg, path)
        distance_travelled = evaluate(path, cities)
        if distance_travelled < min_distance_travelled:
            min_distance_travelled = distance_travelled # update the best solution if the distance is shorter
            best_solution = path
        tracker.append((i, min_distance_travelled, distance_travelled))
    print("algorithm end.")
    end_time = timeit.default_timer()
    print("time taken :{0:.5f}s".format(end_time - start_time))
    
    # display the tracker
    fig = go.Figure()
    x, y, y2 = zip(*tracker)
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='best value'))
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', line={'width': 0.5}, name='iteration value'))
    fig.update_layout(
        title='Best solution found', template="presentation", xaxis_title="Iteration", yaxis_title="Distance")
    fig.show()
    
    return best_solution, min_distance_travelled

In [29]:
# run

our_seed = 100
rg = Generator(PCG64(our_seed)) 
n_cities = 10
cities = simulateCities(rg, n_cities)
solution, distance = FullEnumeration(rg, cities, iterations=5000)
print("best solution:", solution)
print("distance travelled:", distance)

drawSalesman(solution, cities)

running algorithm...
algorithm end.
time taken :0.24077s


best solution: [0, 7, 6, 9, 1, 4, 5, 8, 2, 3, 0]
distance travelled: 2.8392757826878268


In [5]:
def GreedySearch(rg, cities):
    print("running algorithm...")
    start_time = timeit.default_timer() 
    gs_solution = [0]
    n_cities = len(cities)
    unassigned_cities = list(range(1, n_cities))
    while len(unassigned_cities)>0:
        last_city = gs_solution[-1]
        d_map = {i: get_distance(last_city, i, cities) for i in unassigned_cities}
        next_city = min(d_map.items(), key=lambda x: x[1])[0]
        gs_solution.append(next_city)
        unassigned_cities.remove(next_city)
    gs_solution.append(0)  # must return to home
    end_time = timeit.default_timer()
    print("time taken :{0:.5f}s".format(end_time - start_time))
    
    return gs_solution, evaluate(gs_solution, cities)

In [28]:
def apply_2_opt(rg, path):
    new_path = path.copy()
    n_cities =len(new_path)-1
    i1, i2 = 0, 0  
        
    # but, we want them not to be equal zero or equal to each other
    while ((i1==0) or (i2==0) or (i1==i2)):
        i1 = int(np.floor(rg.random() * n_cities))
        i2 = int(np.floor(rg.random() * n_cities))
            
    # perform swap operations
    new_path[i1] = i2
    new_path[i2] = i1
        
    # swap city1 and city2 if city2 > city1
    if i2 > i1:
        temp = i1
        i1 = i2
        i2 = temp
    # so we make sure that city2 < city 1
        
    # reverse the order between these two cities
    j=0
    while j < (i1-i2)/2:
        new_path[i2+j] = path[i1-j]
        new_path[i1-j] = path[i2+j]
        j += 1
    return new_path

def apply_2_opt_v2(path, c2, c1):
    '''new_path = path.copy()
    n_cities =len(new_path)-1

        
    # but, we want them not to be equal zero or equal to each other
    while ((i1==0) or (i2==0) or (i1==i2)):
        i1 = int(np.floor(rg.random() * n_cities))
        i2 = int(np.floor(rg.random() * n_cities))
            
    # perform swap operations
    new_path[i1] = i2
    new_path[i2] = i1
        
    # swap city1 and city2 if city2 > city1
    if i2 > i1:
        temp = i1
        i1 = i2
        i2 = temp
    # so we make sure that city2 < city 1
        
    # reverse the order between these two cities
    j=0
    while j < (i1-i2)/2:
        new_path[i2+j] = path[i1-j]
        new_path[i1-j] = path[i2+j]
        j += 1
    return new_path'''
    new_path = path.copy() # make a copy of path H-1-2-3-...-H
    
    if c1 < c2:
        c1, c2 = c2, c1 # swap c1 and c2 if c1 < c2
    
    new_path[c1] = path[c2] # swap location of c1 and c2
    new_path[c2] = path[c1]
    
    # 2-OPT
    j = 0
    while j < abs(c1-c2)/2: # iterate across inbetween cities
        new_path[c2+j] = path[c1-j]
        new_path[c1-j] = path[c2+j]
        j = j + 1
    
    return new_path

In [7]:
def LocalSearch(rg, cities, iterations):
    tracker = []  # a list of tuple to track the min distance
    path = list(range(len(cities))) + [0]
    distance_travelled = evaluate(path, cities)  # initial objective function value
    
    min_distance_travelled = distance_travelled
    best_solution = path
    
    print("running algorithm...")
    start_time = timeit.default_timer()
    for i in range(1, iterations):
        path = apply_2_opt(rg, path)
        distance_travelled = evaluate(path, cities)
        if distance_travelled < min_distance_travelled:
            min_distance_travelled = distance_travelled
            best_solution = path
        tracker.append((i, min_distance_travelled, distance_travelled))
    print("algorithm end.")
    end_time = timeit.default_timer()
    print("time taken :{0:.5f}s".format(end_time - start_time))
    
    # display the tracker
    fig = go.Figure()
    x, y, y2 = zip(*tracker)
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='best value'))
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', line={'width': 0.5}, name='iteration value'))
    fig.update_layout(
        title='Best solution found', template="presentation", xaxis_title="Iteration", yaxis_title="Distance")
    fig.show()
    
    return best_solution, min_distance_travelled

In [8]:
def temperature_linear(init, i, iterations):
    # linear temperature decrease
    return init * (1-i/iterations)

def temperature_nonlinear(init, i, iterations):
    p_0 = 0.999
    p_f = 0.001
    return -init/np.log(p_0+(p_f-p_0)/iterations*i)

In [9]:
def SimulatedAnnealing(rg, cities, iterations, t_init=1):
    tracker = []
    # path = list(range(len(cities))) + [0]
    path = list(range(len(cities))) + [0]
    distance_travelled = evaluate(path, cities)
    min_distance_travelled = distance_travelled
    best_solution = path
    
    print('running algorithm...')
    start_time = timeit.default_timer()
    state = path # current state
    for i in range(1, iterations):
        path = apply_2_opt(rg, path)  # let's use 2-OPT again
        distance_travelled = evaluate(path, cities)
        # T = temperature_nonlinear(t_init, i, iterations)
        T = temperature_linear(t_init, i, iterations)
        tracker.append((i, min_distance_travelled, distance_travelled))
        d = distance_travelled - min_distance_travelled
        if d<0:
            min_distance_travelled = distance_travelled
            best_solution = path
            state = path
        else:
            if rg.random() < np.exp(-d/T):
                state = path # only update state
    print('algorithm end.')
    end_time = timeit.default_timer()
    print("time taken :{0:.5f}s".format(end_time - start_time))
    
    # display the tracker
    fig = go.Figure()
    x, y, y2 = zip(*tracker)
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='best value'))
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', line={'width': 0.5}, name='iteration value'))
    fig.update_layout(
        title='Best solution found', template="presentation", xaxis_title="Iteration", yaxis_title="Distance")
    fig.show()
    
    return best_solution, min_distance_travelled      

In [17]:
from plotly.subplots import make_subplots
def VNS(rg, cities, iterations, K=5):
    tracker = []
    path = list(range(len(cities))) + [0]
    distance_travelled = evaluate(path, cities)
    min_distance_travelled = distance_travelled
    best_solution = path
    print('running algorithm...')
    start_time = timeit.default_timer()
    k=0 # initialize
    for i in range(1, iterations):
        k = k+1
        c2 = rg.choice(range(1, len(cities)-1))  # pick a random city
        
        # define the neighborhood structure
        N = list(range(c2, c2+1+k))
        neighbours = [n for n in N if (n>0 and n<len(cities) and n!=c2)]
        
        for c1 in neighbours: # step through each neighbor one at a time
            path = apply_2_opt_v2(path, c2, c1)
            distance_travelled = evaluate(path, cities)
            tracker.append((i, min_distance_travelled, distance_travelled, k))
            
            if distance_travelled < min_distance_travelled:
                min_distance_travelled = distance_travelled
                best_solution = path
                k = 0
                break
        if k == K:
            print('k reached')
            break
    print('algorithm end.')
    end_time = timeit.default_timer()
    print("time taken :{0:.5f}s".format(end_time - start_time))
    
    # display the tracker
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    x, y, y2, k = zip(*tracker)
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='best value'), secondary_y=False)
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', line={'width': 0.5}, name='iteration value'), secondary_y=False)
    fig.add_trace(go.Scatter(x=x, y=k, mode='lines', name='neighbourhood size'), secondary_y=True)
    fig.update_layout(
        title='Best solution found', template="presentation", xaxis_title="Iteration", yaxis_title="Distance")
    fig.show()
    
    return best_solution, min_distance_travelled    

In [30]:
def OptimizationTSPTest(rg, iterations=5000):
    
    cities = simulateCities(rg, n_cities)
    inital_solution = list(range(len(cities))) + [0]
    print(cities)
    print("initial solution:", inital_solution)
    print("inital distance travelled:", evaluate(inital_solution, cities))
    drawSalesman(inital_solution, cities, "Initial Solution") # show the inital solution
    
    # Full enumeration section 
    print("Full Enumeration\n-----------")
    fe_solution, fe_distance = FullEnumeration(rg, cities, iterations=iterations)
    print("best solution:", solution)
    print("distance travelled:", fe_distance)
    drawSalesman(fe_solution, cities, "Full enumeration")
    
    print('\n\n\n')
    
    # Greedy search algorithm section
    # Implement your greedy algorithm here
    print("Greedy Search:\n-----------")
    gs_solution, gs_distance = GreedySearch(rg, cities)
    drawSalesman(gs_solution, cities, "Greedy")
    print("greedy solution:", gs_solution)
    print("distance travelled:", gs_distance)
    
    print('\n\n\n')
    
    # Local search
    print('Local Search\n------------')
    ls_solution, ls_distance = LocalSearch(rg, cities, iterations)
    drawSalesman(ls_solution, cities, "Local Search")
    print("best solution:", ls_solution)
    print("distance travelled:", ls_distance)
    
    print('\n\n\n')
    
    # Simulated annealing
    print('Simulated Annealing\n----------------')
    sa_solution, sa_distance = SimulatedAnnealing(rg, cities, iterations)
    drawSalesman(sa_solution, cities, "Simulated Annealing")
    print("best solution:", sa_solution)
    print("distance travelled:", sa_distance)
    
    print('\n\n\n')
    
    # VNS
    print('Variable Neighborhood Search\n----------------')
    vns_solution, vns_distance = VNS(rg, cities, iterations, K=50)
    drawSalesman(vns_solution, cities, "VNS")
    print("best solution:", vns_solution)
    print("distance travelled:", vns_distance)
    
    print('\n\n\n')

# run all
rg = Generator(PCG64(our_seed)) # use your own seed number here
OptimizationTSPTest(rg, iterations=5000)

{0: (0.83, 0.6), 1: (0.29, 0.04), 2: (0.97, 0.6), 3: (0.79, 0.91), 4: (0.69, 0.19), 5: (0.98, 0.28), 6: (0.63, 0.58), 7: (0.6, 0.54), 8: (1.0, 0.5), 9: (0.77, 0.49)}
initial solution: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
inital distance travelled: 4.316368582016129


Full Enumeration
-----------
running algorithm...
algorithm end.
time taken :0.24444s


best solution: [0, 7, 6, 9, 1, 4, 5, 8, 2, 3, 0]
distance travelled: 2.8392757826878268






Greedy Search:
-----------
running algorithm...
time taken :0.00032s


greedy solution: [0, 9, 6, 7, 4, 5, 8, 2, 3, 1, 0]
distance travelled: 3.4719328943942944




Local Search
------------
running algorithm...
algorithm end.
time taken :0.19531s


best solution: [0, 2, 3, 8, 5, 4, 1, 6, 7, 9, 0]
distance travelled: 2.901494953123203




Simulated Annealing
----------------
running algorithm...
algorithm end.
time taken :0.21459s


best solution: [0, 3, 7, 6, 1, 4, 5, 9, 8, 2, 0]
distance travelled: 2.919074842496755




Variable Neighborhood Search
----------------
running algorithm...
k reached
algorithm end.
time taken :0.03673s


best solution: [0, 8, 5, 4, 1, 7, 3, 2, 9, 6, 0]
distance travelled: 3.107371988903894






# Lab 7 Optimization exercises: Multi-objective optimization

Implement the multi-objective optimization on the Knapsack problem using your choice of algorithm:
- simulated annealing
- variable neighbourhood search
- local search

## Knapsack problem

Each item has a weight $w$, utility $u$ and value $c$

objective functions:

$x_i=1$ if $i$ in knapsack, else $0$.

1. $\max \mathcal{f}(x)=\sum_{i=1}^n u_i x_i$ 
2. $\min \mathcal{f}(x)=\sum_{i=1}^n c_i x_i$
3. $w^Tx \leq W$ where $W$ is the maximum weight allowed: **150kg**

What is the set of items that can be put in the knapsack that is below the maximum weight and maximizes value and utility?

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd

from numpy.random import Generator, PCG64

In [ ]:
# objective functions
def total_value(item_value, solution):
    return np.sum(item_value[solution])

def total_utility(item_utility, solution):
    return np.sum(item_utility[solution])

# constraints on weight, returns True if items below max_weight, else false
def weight_limit(item_weights, solution, max_weight):
    total_weight = np.sum(item_weights[solution])
    return total_weight < max_weight

def weight(item_weights, solution):
    return np.sum(item_weights[solution])

In [ ]:
# cost is a tuple (value, utility)
cost = (total_value(item_value, solution), total_utility(item_utility, solution))
    
# add to P as a dictionary of values
P.append({'solution': solution, 'cost': cost, 'weight': weight(item_weights, solution)}) 

NameError: name 'item_value' is not defined

In [ ]:
def maximize(old_value, new_value):
    return old_value <= new_value

def minimize(old_value, new_value):
    return new_value <= old_value

def maximize_strict(old_value, new_value):
    return old_value < new_value

def minimize_strict(old_value, new_value):
    return new_value < old_value


In [ ]:
# Pareto dominance
def improvement(cost_old, cost_new):
    """
    
    Iput
    -------------
    cost_old: (value_old, utility_old)
    cost_new: (value_new, utility_new)
    
    """
    is_dominating = 0 
    
    rule_1 = []
    rule_2 = []
    
    value_old, utility_old = cost_old
    value_new, utility_new = cost_new
    
  
    rule_1.append(minimize(value_old, value_new))
    rule_1.append(maximize(utility_old, utility_new))
    
    rule_2.append(minimize_strict(value_old, value_new))
    rule_2.append(maximize_strict(utility_old, utility_new))
    
    if all(rule_1) and any(rule_2):
        is_dominating = 1
    
    return is_dominating

def dominance(P, cost_new):
    
    # D is the list of solutions in the frontier set
    # which are dominated by the new solution
    D = []
    
    # S is the list of solutions in the frontier set
    # which dominates the current solution
    S = []
    
    for i, p in enumerate(P):
        cost = p['cost']
        if improvement(cost, cost_new):
            D.append(i)
        if improvement(cost_new, cost):
            S.append(i)
    return D, S

In [ ]:
# Example: full enumeration algorithm

def knapsack_fe(rg, solution, item_weights, W):
    n = rg.choice(range(1, 20)) # random number of items (up to 20 items)
    new_solution = rg.choice(range(20), n, replace=False) # pick n items from [1,...,20]

    while not weight_limit(item_weights, new_solution, W): # pick another solution if new_solution is above limit
        n = rg.choice(range(1, 20))
        new_solution = rg.choice(range(20), n, replace=False)
    
    return new_solution

In [ ]:
def select_neighbor(solution: list, number_of_items: int, k: int)->list:
    """
    solution, current solution
    number_of_items, size of the items list
    k, neighbors size
    
    returns a new proposed solution
    """
    
    # randomly select a neighbor x^+
    neighborhood = np.arange(0, number_of_items, step=1)
    neighborhood = rg.choice(neighborhood, k, replace=False)
    
    # change decision
    # if x in solution 1->0
    # otherwise 0->1
    new_solution = [x for x in neighborhood if x not in solution]
    return new_solution

def knapsack_vns(rg: PCG64, solution: list, item_weights: int, item_utility: list, W: int, k: int)->list:
    """
    rg, the random number generator
    solution, current solution
    item_weights, list storing each item weights,
    item_utility, list storing each item utility
    W, the max_weight,
    k, size of neighbors
    
    Slide 73 07optimization.pdf
    """
    number_of_items = len(item_weights)
    new_solution = select_neighbor(solution, number_of_items, k)
    
    # u^T*x_c
    current_utility = total_utility(item_utility, solution)
    
    # u^T*x^+
    new_utility = total_utility(item_utility, new_solution)
    
    if weight_limit(item_weights, new_solution, W) and new_utility > current_utility:
        solution = new_solution

    # repeat 1000 times
    i = 0
    while i<1000:

        
        new_solution = select_neighbor(solution, number_of_items, k)

        # u^T*x_c
        current_utility = total_utility(item_utility, solution)

        # u^T*x^+
        new_utility = total_utility(item_utility, new_solution)

        if weight_limit(item_weights, new_solution, W) and new_utility > current_utility:
            solution = new_solution

        i += 1
    
    return solution

In [ ]:
def main(rg, iterations=1000, W=150):
    
    # Given Default: 20 items
    # Wong: simulate some random values
    item_weights = rg.integers(10,50,20)
    item_utility = rg.integers(0,100,20)
    item_value = rg.random(20)
    
    # P is a list of dictionaries -> Pareto solution
    P = [] # Wong: initialize empty lists to store solutions and costs
    
    # Q is a list of dictionaries -> list of considered
    Q = []
    
    # R is a list of dictionaries -> list of removed
    R = []
    
    # Wong: generate an initial random solution and add it to P
    # Wong: example start with 2 random item from the set
    
    # solution is a list of items INDEX
    solution = list(rg.choice(len(item_weights), 2, replace=False)) 
    
    # cost is a tuple (value, utility)
    cost = (total_value(item_value, solution), total_utility(item_utility, solution))
    
    # add to P as a dictionary of values
    P.append({'solution': solution, 'cost': cost, 'weight': weight(item_weights, solution)}) 
    
    # Main iteration
    for i in range(1, iterations):
        
        # select a random solution set from set P
        p = rg.choice(P)
        
        #################################
        # Implement your algorithm here #
        #################################
        #new_solution = knapsack_fe(rg, solution, item_weights, W)
        new_solution = knapsack_vns(rg, solution, item_weights, item_utility, W, k=4)
        #################################
        
        # compute the new cost of the new solution
        new_cost = (total_value(item_value, new_solution), total_utility(item_utility, new_solution))
        
        # Pareto dominance algorithm
        D, S = dominance(P, new_cost)
        # Pareto algorithm update
        if len(S) == 0:
            # add items p in P to list R for all items in D and not already in R
            R.extend([p for n, p in enumerate(P) if (n in D and p not in R)])
            # remove items in P
            P = [p for n, p in enumerate(P) if n not in D]
            # finally add new_solution to P
            P.append({'solution': list(new_solution), 'cost': new_cost, 'weight': weight(item_weights, new_solution)})
        else:
            # track considered solutions
            Q.append({'solution': list(new_solution), 'cost': new_cost, 'weight': weight(item_weights, new_solution)})
            if len(Q) > 100:
                Q.pop(0) # we store only the last 100 considered values for memory savings
    
    print('optimization complete')
    # return the pareto set, considered set and removed set
    return P, Q, R

# run
rg = Generator(PCG64(42069)) # set your own unique seed number
P, Q, R = main(rg, iterations=1000, W=150)

Iteration Number: 1
Iteration Number: 2
Iteration Number: 3
Iteration Number: 4
Iteration Number: 5
Iteration Number: 6
Iteration Number: 7
Iteration Number: 8
Iteration Number: 9
Iteration Number: 10
Iteration Number: 11
Iteration Number: 12
Iteration Number: 13
Iteration Number: 14
Iteration Number: 15
Iteration Number: 16
Iteration Number: 17
Iteration Number: 18
Iteration Number: 19
Iteration Number: 20
Iteration Number: 21
Iteration Number: 22
Iteration Number: 23
Iteration Number: 24
Iteration Number: 25
Iteration Number: 26
Iteration Number: 27
Iteration Number: 28
Iteration Number: 29
Iteration Number: 30
Iteration Number: 31
Iteration Number: 32
Iteration Number: 33
Iteration Number: 34
Iteration Number: 35
Iteration Number: 36
Iteration Number: 37
Iteration Number: 38
Iteration Number: 39
Iteration Number: 40
Iteration Number: 41
Iteration Number: 42
Iteration Number: 43
Iteration Number: 44
Iteration Number: 45
Iteration Number: 46
Iteration Number: 47
Iteration Number: 48
I

Iteration Number: 381
Iteration Number: 382
Iteration Number: 383
Iteration Number: 384
Iteration Number: 385
Iteration Number: 386
Iteration Number: 387
Iteration Number: 388
Iteration Number: 389
Iteration Number: 390
Iteration Number: 391
Iteration Number: 392
Iteration Number: 393
Iteration Number: 394
Iteration Number: 395
Iteration Number: 396
Iteration Number: 397
Iteration Number: 398
Iteration Number: 399
Iteration Number: 400
Iteration Number: 401
Iteration Number: 402
Iteration Number: 403
Iteration Number: 404
Iteration Number: 405
Iteration Number: 406
Iteration Number: 407
Iteration Number: 408
Iteration Number: 409
Iteration Number: 410
Iteration Number: 411
Iteration Number: 412
Iteration Number: 413
Iteration Number: 414
Iteration Number: 415
Iteration Number: 416
Iteration Number: 417
Iteration Number: 418
Iteration Number: 419
Iteration Number: 420
Iteration Number: 421
Iteration Number: 422
Iteration Number: 423
Iteration Number: 424
Iteration Number: 425
Iteration 

Iteration Number: 757
Iteration Number: 758
Iteration Number: 759
Iteration Number: 760
Iteration Number: 761
Iteration Number: 762
Iteration Number: 763
Iteration Number: 764
Iteration Number: 765
Iteration Number: 766
Iteration Number: 767
Iteration Number: 768
Iteration Number: 769
Iteration Number: 770
Iteration Number: 771
Iteration Number: 772
Iteration Number: 773
Iteration Number: 774
Iteration Number: 775
Iteration Number: 776
Iteration Number: 777
Iteration Number: 778
Iteration Number: 779
Iteration Number: 780
Iteration Number: 781
Iteration Number: 782
Iteration Number: 783
Iteration Number: 784
Iteration Number: 785
Iteration Number: 786
Iteration Number: 787
Iteration Number: 788
Iteration Number: 789
Iteration Number: 790
Iteration Number: 791
Iteration Number: 792
Iteration Number: 793
Iteration Number: 794
Iteration Number: 795
Iteration Number: 796
Iteration Number: 797
Iteration Number: 798
Iteration Number: 799
Iteration Number: 800
Iteration Number: 801
Iteration 

## Display the output

In [ ]:
fig = go.Figure()


c = pd.DataFrame(P)
x, y = list(zip(*c['cost']))
fig.add_trace(go.Scatter(x=x, y=y, mode='markers+text', name='pareto', 
    hovertemplate ='items: [%{hovertext}]', hovertext=c['solution'],
    text=c['weight'], texttemplate = "%{text} kg", textposition="top center", textfont = {'size': 12}))

c = pd.DataFrame(R)
x, y = list(zip(*c['cost']))
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='removed',
    hovertemplate ='items: [%{hovertext}]', hovertext=c['solution'],))

c = pd.DataFrame(Q)
x, y = list(zip(*c['cost']))
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='considered',
    hovertemplate ='items: [%{hovertext}]', hovertext=c['solution'],))

fig.update_layout(template="presentation", xaxis_title="Value", yaxis_title="Utility", width=800, height=600)
fig.show()